In [68]:
from typing import List

class TrieNode:
    def __init__(self):
        self.children = [None] * 26
        self.isWord = 0 # use int rather than bool to count duplicate words
        self.suggestions = []
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    def __getIdx(self, ch: str) -> int:
        return ord(ch) - ord('a')
    
    def __toChr(self, i: int) -> str:
        return chr(i+97)
    
    def insert(self, word: str):
        node = self.root
        for ch in word:
            i = self.__getIdx(ch)
            if not node.children[i]:
                node.children[i] = TrieNode()
            node = node.children[i]
        node.isWord += 1
        
    def startsWith(self, prefix: str) -> List[str]:
        node = self.root
        res = []
        for ch in prefix:
            i = self.__getIdx(ch)
            if not node.children[i]:
                return res
            node = node.children[i]
        words = self.dfs(node, list(prefix))
        return words
        
    def dfs(self, node: TrieNode, path: List[str], k: int = 3):
        words = []
        if not node:
            return []
        if node.suggestions:
            return node.suggestions
        if node.isWord:
            w = ''.join(path)
            words += [w] * node.isWord
        for i, c in enumerate(node.children):
            ws = self.dfs(c, path + [self.__toChr(i)], k)
            words += ws
            if len(words) >= k:
                node.suggestions = words[:k]
                return node.suggestions
        node.suggestions = words
        return node.suggestions

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        trie = Trie()
        for w in products:
            trie.insert(w)
        return [trie.startsWith(searchWord[:i]) for i in range(1, len(searchWord)+1)]

In [120]:
# Optimized Trie solution
# 1. use max-heap to store top 3 suggestions in trie node (memorization)
# 2. record last typed char's TrieNode in searchWord, which saves traversing from root everytime
# Runtime: 332 ms
# https://leetcode.com/problems/search-suggestions-system/discuss/436183/Python-Trie-Solution
from typing import List

class TrieNode:
    def __init__(self):
        self.children = [None] * 26
        self.suggestions = []
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    def __getIdx(self, ch: str) -> int:
        return ord(ch) - ord('a')
    
    def insert(self, word: str):
        node = self.root
        for ch in word:
            i = self.__getIdx(ch)
            if not node.children[i]:
                node.children[i] = TrieNode()
            node = node.children[i]
            # for every node in the path (prefix node), add this word to its suggestions
            # max_heap is another solution besides array sorting
            node.suggestions.append(word)
            node.suggestions = sorted(node.suggestions)[:3]
        
    def startsWith(self, prefix: str) -> List[str]:
        node = self.root
        res = []
        for ch in prefix:
            suggestions, node = self.nextNode(node, ch)
            res.append(suggestions)
        return res
    
    def nextNode(self, node: TrieNode, ch: str) -> (List[str], TrieNode):
        if not node: return ([], None)
        i = self.__getIdx(ch)
        if not node.children[i]:
            return ([], None)
        node = node.children[i]
        return (node.suggestions, node)

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        trie = Trie()
        for w in products:
            trie.insert(w)
        return trie.startsWith(searchWord)

In [138]:
# Binary Search
# Runtime: 64 ms
from typing import List
from bisect import *

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        products.sort()
        L = len(products)
        res = []
        for i in range(1, len(searchWord)+1):
            # find first prefix match idx
            sw = searchWord[:i]
            idx = bisect_left(products, sw)
            # scan for another 3 elements - check match
            res.append([w for w in products[idx:idx+3] if w.find(sw) == 0])
#             matches = []
#             j = 0
#             while idx < L and products[idx].find(searchWord[:i]) == 0 and j < 3:
#                 matches.append(products[idx])
#                 idx += 1
#                 j += 1
#             res.append(matches)
        return res

In [139]:
Solution().suggestedProducts(products = ["mobile","mouse","moneypot","monitor","mousepad"], searchWord = "mouse")

[['mobile', 'moneypot', 'monitor'],
 ['mobile', 'moneypot', 'monitor'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad']]

In [140]:
# duplicates
Solution().suggestedProducts(products = ["mobile","mouse","moneypot","monitor","mousepad", "mase", "mase", "mouseapple"], searchWord = "mouse")

[['mase', 'mase', 'mobile'],
 ['mobile', 'moneypot', 'monitor'],
 ['mouse', 'mouseapple', 'mousepad'],
 ['mouse', 'mouseapple', 'mousepad'],
 ['mouse', 'mouseapple', 'mousepad']]

In [141]:
Solution().suggestedProducts(products = ["havana"], searchWord = "havana")

[['havana'], ['havana'], ['havana'], ['havana'], ['havana'], ['havana']]

In [142]:
Solution().suggestedProducts(products = ["bags","baggage","banner","box","cloths"], searchWord = "bags")

[['baggage', 'bags', 'banner'],
 ['baggage', 'bags', 'banner'],
 ['baggage', 'bags'],
 ['bags']]

In [143]:
Solution().suggestedProducts(products = ["havana"], searchWord = "tatiana")

[[], [], [], [], [], [], []]